In [ ]:
pip install opencv-python deepface streamlit spotipy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.5/261.5 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.5 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=6bfe0f0b85d5aa57393d10da2a54d23874f1cb94aa963836972d0af2da0b63ee
  Stored in directory: /roo

In [ ]:
pip install opencv-python deepface gradio matplotlib spotipy pillow numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 3.6 MB/s eta 0:00:00


In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from IPython.display import Image
import PIL.Image
import io
import cv2
import numpy as np

def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
        async function takePhoto(quality) {
          const div = document.createElement('div');
          const capture = document.createElement('button');
          capture.textContent = '📸 Capture';
          div.appendChild(capture);

          const video = document.createElement('video');
          video.style.display = 'block';
          const stream = await navigator.mediaDevices.getUserMedia({video: true});

          document.body.appendChild(div);
          div.appendChild(video);
          video.srcObject = stream;
          await video.play();

          // Resize the output to fit the video element.
          google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

          // Wait for Capture to be clicked.
          await new Promise((resolve) => capture.onclick = resolve);

          const canvas = document.createElement('canvas');
          canvas.width = video.videoWidth;
          canvas.height = video.videoHeight;
          canvas.getContext('2d').drawImage(video, 0, 0);
          stream.getVideoTracks()[0].stop();
          div.remove();

          const dataUrl = canvas.toDataURL('image/jpeg', quality);
          return dataUrl;
        }
    ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = io.BytesIO(base64.b64decode(data.split(',')[1]))
    img = PIL.Image.open(binary)
    img.save(filename)
    return filename


In [ ]:
import base64

filename = take_photo()
print('Saved to {}'.format(filename))

# Display image
img = cv2.imread(filename)
cv2.imwrite("test.jpg", img)

<IPython.core.display.Javascript object>

Saved to photo.jpg


True

In [ ]:
import cv2
import numpy as np
import gradio as gr
import matplotlib.pyplot as plt
from deepface import DeepFace
import base64
from PIL import Image
import io
import os

# Emotion to genre mapping
emotion_to_genre = {
    'happy': 'pop chill',
    'sad': 'lofi chill',
    'angry': 'hard rock',
    'surprise': 'edm',
    'neutral': 'instrumental',
    'fear': 'calming ambient',
    'disgust': 'indie'
}

# Spotify links for each emotion
spotify_links = {
    'happy': 'https://open.spotify.com/playlist/37i9dQZF1DXdPec7aLTmlC',  # Happy Hits!
    'sad': 'https://open.spotify.com/playlist/37i9dQZF1DWVrtsSlLKzro',    # Life Sucks
    'angry': 'https://open.spotify.com/playlist/37i9dQZF1DX3qCx5yEZkcJ',   # Rock Hard
    'neutral': 'https://open.spotify.com/playlist/37i9dQZF1DWT0upuUFtT7o', # Chill Hits
    'fear': 'https://open.spotify.com/playlist/37i9dQZF1DWVIzZt2GAU4X',    # Deep Focus
    'disgust': 'https://open.spotify.com/playlist/37i9dQZF1DX6VdMW310YC7', # Indie Chill
    'surprise': 'https://open.spotify.com/playlist/37i9dQZF1DWTwnEm1IYyoj' # Electro Mix
}

# YouTube search URLs
def get_youtube_url(genre):
    search_query = f"{genre} playlist"
    return f"https://www.youtube.com/results?search_query={search_query.replace(' ', '+')}"

# Emoji mapping
emoji_map = {
    'happy': '😄',
    'sad': '😢',
    'angry': '😡',
    'surprise': '😮',
    'neutral': '😐',
    'fear': '😨',
    'disgust': '🤢'
}

def analyze_emotion(image):
    if image is None:
        return "No image provided. Please upload an image.", None, None

    # Save the image temporarily
    try:
        if isinstance(image, np.ndarray):
            cv2.imwrite("temp_capture.jpg", cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
        else:
            # If it's a file path
            pil_image = Image.open(image)
            pil_image.save("temp_capture.jpg")

        img_path = "temp_capture.jpg"

        # Analyze the image
        result = DeepFace.analyze(img_path=img_path, actions=['emotion'])
        emotion = result[0]['dominant_emotion']
        emoji = emoji_map.get(emotion, '')

        # Get genre based on emotion
        genre = emotion_to_genre.get(emotion, 'relaxing music')

        # Get spotify link
        spotify_url = spotify_links.get(emotion, 'https://open.spotify.com/playlist/37i9dQZF1DX4WYpdgoIcn6')

        # Get YouTube link
        youtube_url = get_youtube_url(genre)

        # Create HTML with clickable links
        html_result = f"""
        <div style="font-family: Arial, sans-serif; padding: 15px; border-radius: 10px; background-color: #f9f9f9;">
            <h3>Detected Emotion: {emotion.capitalize()} {emoji}</h3>
            <p>Recommended Genre: <b>{genre.title()}</b></p>
            <div style="margin-top: 15px;">
                <a href="{spotify_url}" target="_blank" style="display: inline-block; padding: 10px 15px; margin-right: 10px; text-decoration: none; color: white; background-color: #1DB954; border-radius: 5px;">
                    🎧 Open Spotify Playlist
                </a>
                <a href="{youtube_url}" target="_blank" style="display: inline-block; padding: 10px 15px; text-decoration: none; color: white; background-color: #FF0000; border-radius: 5px;">
                    🎬 Search on YouTube
                </a>
            </div>
            <p style="margin-top: 15px; font-size: 12px;">Click on the buttons above to open the music platforms in a new tab</p>
        </div>
        """

        # Read the image for display
        if isinstance(image, np.ndarray):
            display_img = image if image.shape[2] == 3 else cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        else:
            display_img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)

        # Create a visualization
        plt.figure(figsize=(10, 8))
        plt.imshow(display_img)
        plt.axis('off')
        plt.title(f"Detected Emotion: {emotion.capitalize()} {emoji}")

        # Save the visualization
        plt.savefig("emotion_detection.jpg")
        plt.close()

        # Also provide plain text for non-HTML environments
        plain_text_result = f"""
Detected Emotion: {emotion.capitalize()} {emoji}
Recommended Genre: {genre.title()}

Spotify Playlist: {spotify_url}
YouTube Search: {youtube_url}

Copy and paste these links into your browser to open them.
"""

        return html_result, "emotion_detection.jpg", plain_text_result

    except Exception as e:
        return f"Error processing image: {str(e)}", None, None

# Main Gradio interface
def html_links_interface():
    with gr.Blocks(title="Emotion-Based Music Recommender") as app:
        gr.Markdown("# 😊 Emotion-Based Music Recommender 🎵")
        gr.Markdown("Upload a photo, and get music recommendations based on your emotion!")

        # Input
        with gr.Row():
            input_image = gr.Image(label="Upload or Take a Photo")

        # Analyze button
        with gr.Row():
            analyze_btn = gr.Button("Analyze Emotion", variant="primary")

        # Outputs
        with gr.Row():
            with gr.Column():
                # HTML output with clickable links
                html_output = gr.HTML(label="Results")
                # Backup text output (in case HTML doesn't work)
                text_output = gr.Textbox(label="Links (Copy & Paste)", visible=True, lines=6)
            with gr.Column():
                output_image = gr.Image(label="Emotion Detection Results")

        # Set up event handler
        analyze_btn.click(
            fn=analyze_emotion,
            inputs=[input_image],
            outputs=[html_output, output_image, text_output]
        )

        gr.Markdown("""
        ## How it works
        1. Upload or take a photo
        2. Click "Analyze Emotion" to detect your mood
        3. Get music recommendations based on your emotion
        4. Click the provided links to open Spotify or YouTube

        ## Emotion to Music Genre Mapping
        - Happy → Pop Chill
        - Sad → Lofi Chill
        - Angry → Hard Rock
        - Surprise → EDM
        - Neutral → Instrumental
        - Fear → Calming Ambient
        - Disgust → Indie
        """)

    return app

# Launch the app
if __name__ == "__main__":
    app = html_links_interface()
    app.launch(share=True)  # Creates a public link

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://dc7ff553302c2e1cae.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
